```json
  {  "Name": "Portland, OR",

  "Lat": "45.59578°N",

  "Lon": "122.60917°W",

  "Elev": "20ft.",

  "Temperature": "45\u00b0F",

  "Humidity": "75%",

  "Wind Speed": "WSW 8 mph",

  "Barometer": "29.99 in (1015.58 mb)",

  "Dewpoint": "37\u00b0F (3\u00b0C)",

  "Visibility": "10.00 mi",

  "Wind Chill": "41\u00b0F (5\u00b0C)",

  "Last update": "13 Feb 10:15 AM PST"}



  {  "Name": "San Diego, CA",

  "Lat": "32.73361°N",

  "Lon": "117.18306°W",

  "Elev": "13ft.",

  "Temperature": "55\u00b0F",

  "Humidity": "66%",

  "Wind Speed": "SE 3 MPH",

  "Barometer": "30.0 in (1015.92 mb)",

  "Dewpoint": "44\u00b0F (7\u00b0C)",

  "Visibility": "10.00 mi",

  "Wind Chill": "Null",

  "Last update": "13 Feb 09:35 AM PST"}



  {  "Name": "Duluth, MN",

  "Lat": "46.72°N",

  "Lon": "92.04°W",

  "Elev": "607ft.",

  "Temperature": "39\u00b0F",

  "Humidity": "56%",

  "Wind Speed": "SW 10 mph",

  "Barometer": "27.71 in (1029.8 mb)",

  "Dewpoint": "25\u00b0F (-4\u00b0C)",

  "Visibility": "10.00 mi",

  "Wind Chill": "32\u00b0F (0\u00b0C)",

  "Last update": "13 Feb 12:55 pm CST"}
```

In [ ]:
import json
import nws_scraper_utils
from selenium import webdriver
import time

time_stamp = time.strftime("%Y%m%d-%H%M%S")
time_stamp_filename = time_stamp + '_nws_log.json'

driver = webdriver.Firefox()

'''--------- Define CityCurrentStats class --------'''

class CityObject:
    def __init__(self, name, url, driver):
        self.name = {'Name': name}
        self.url = {'URL' : url}
        self.temperature = nws_scraper_utils.get_temperature(url, driver)
        self.condition_description = nws_scraper_utils.get_condition_description(url, driver)
        self.station = nws_scraper_utils.get_station(url, driver)
        self.condition_details = nws_scraper_utils.get_condition_details(url, driver)

def main():

    ''' read the user input file in json format and convert to dict name:url'''
    try:
        data_input_file = 'cities.json'  # read user input file
        with open(data_input_file) as json_data:  # This returns a list of dictionaries
            cities = json.load(json_data)
    except FileNotFoundError:
        print('quitting, can\'t input file ', data_input_file)
        exit()


    '''for each dict in list , create a CityObject and populate it from NWS data'''
    for city_pair in cities:                # This iterates and returns individual dictionaries from the list
        name = (city_pair['Name'])          # and we extract the value from each key here...
        url = (city_pair['NWS_URL'])
        city = CityObject(name,url,driver)         # create a City Object
        all_info = nws_scraper_utils.merge_dictionaries(city.name, city.temperature, city.condition_details)


        # Finally, append each city dictionary to json file
        # with a time-stamped filename
        final_output_json = json.dumps(all_info, indent = 4)
        # Open new json file if not exist it will create
        fp = open(time_stamp_filename, 'a')  # append
        # write to json file
        fp.write(final_output_json)
        print(final_output_json)
    # close the output file once IN THE OUTER LOOP
    fp.close()

    driver.close()
    driver.quit()